<a href="https://colab.research.google.com/github/zerotodeeplearning/ztdl-masterclasses/blob/master/notebooks/Image_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Copyright 2020 Catalit LLC.

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Image Search

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import os
from tensorflow.keras.preprocessing import image

In [ ]:
# sports_images_path = tf.keras.utils.get_file(
#     'sports_images',
#     'https://archive.org/download/ztdl_sports_images/sports_images.tgz',
#      untar=True)

In [ ]:
![[ ! -f sports_images.tar.gz ]] && gsutil cp gs://ztdl-datasets/sports_images.tar.gz .
![[ ! -d sports_images ]] && echo "Extracting images..." && tar zxf sports_images.tar.gz
sports_images_path  = './sports_images'

In [ ]:
train_path = os.path.join(sports_images_path, 'train')
test_path = os.path.join(sports_images_path, 'test')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input

In [ ]:
img_size = 299

In [ ]:
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
bottleneck_generator = datagen.flow_from_directory(
    train_path,
    target_size=(img_size, img_size),
    batch_size=2000,
    class_mode = 'sparse',
    shuffle=True)

In [ ]:
batch, labels = bottleneck_generator.next()

In [ ]:
base_model = Xception(include_top=False,
                      weights='imagenet',
                      input_shape=(img_size, img_size, 3),
                      pooling='avg')

In [ ]:
bottlenecks = base_model.predict(batch, verbose=1)

In [ ]:
from sklearn.neighbors import DistanceMetric

In [ ]:
dist = DistanceMetric.get_metric('euclidean')

In [ ]:
distance_matrix = dist.pairwise(bottlenecks)

In [ ]:
def imshow_scaled(img):
    plt.imshow((img + 1) / 2)

def image_search(img_index, n_rows=3, n_columns=3):
    n_images = n_rows * n_columns

    dist_from_sel = pd.Series(distance_matrix[img_index])
    retrieved = dist_from_sel.sort_values().head(n_images)
    
    plt.figure(figsize=(10, 10))
    i = 1
    for idx in retrieved.index:
        plt.subplot(n_rows, n_columns, i)
        imshow_scaled(batch[idx])
        if i == 1:
            plt.title('Selected image')
        else:
            plt.title("Dist: {:0.4f}".format(retrieved[idx]))
        i += 1
        plt.axis('off')
    plt.tight_layout()

In [ ]:
image_search(0)

### Exercise 1

- Experiment with the `image_search` function. Change the selected image, are the other images found similar?
- Sort the distance matrix by label and visualize it using `plt.imshow` can you see blocks appear?